In [ ]:
"""
Generate transactions.csv for Data Science Course
==================================================
Uses Faker library for realistic transaction data
Includes seasonality, customer behavior patterns, and time-series data

Run: pip install faker pandas numpy
Note: Requires customers.csv and products.csv to be generated first
"""

import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import time

# Initialize
fake = Faker()
Faker.seed(42)
np.random.seed(42)

print("="*70)
print("GENERATING TRANSACTIONS.CSV")
print("="*70)

# Configuration
NUM_TRANSACTIONS = 100000
START_DATE = datetime(2019, 1, 1)
END_DATE = datetime(2024, 12, 31)

start_time = time.time()
print(f"\nGenerating {NUM_TRANSACTIONS:,} transactions...")

# ============================================================================
# LOAD PREREQUISITE DATA
# ============================================================================

print("  → Loading customers and products data...")

try:
    customers_df = pd.read_csv('customers.csv')
    products_df = pd.read_csv('products.csv')
    print(f"     ✓ Loaded {len(customers_df):,} customers")
    print(f"     ✓ Loaded {len(products_df):,} products")
except FileNotFoundError as e:
    print(f"\n❌ ERROR: {e}")
    print("Please run generate_customers.py and generate_products.py first!")
    exit(1)

customer_ids = customers_df['Customer_ID'].values
product_ids = products_df['Product_ID'].values

# ============================================================================
# CUSTOMER BEHAVIOR PATTERNS
# ============================================================================

print("  → Modeling customer behavior patterns...")

# Create customer purchase propensity (some customers buy more frequently)
# Using Zipf distribution - power law (realistic for e-commerce)
customer_activity = {}
for cid in customer_ids:
    # Activity level: 1-10 (higher = more active)
    activity = np.random.zipf(a=2)
    activity = min(activity, 10)
    customer_activity[cid] = activity

# Convert to weights for sampling
total_activity = sum(customer_activity.values())
customer_weights = np.array([customer_activity[cid] for cid in customer_ids])
customer_weights = customer_weights / customer_weights.sum()

# ============================================================================
# PRODUCT POPULARITY
# ============================================================================

print("  → Calculating product popularity...")

# Products with higher ratings and more reviews are more popular
product_popularity = {}
for _, row in products_df.iterrows():
    pid = row['Product_ID']
    rating = row['Rating']
    reviews = row['Number_of_Reviews']
    stock = row['Stock_Quantity']
    
    # Popularity score
    if stock == 0:
        popularity = 0.01  # Very low but not zero (backorders possible)
    else:
        # Higher rating and more reviews = more popular
        popularity = (rating ** 2) * (1 + np.log1p(reviews))
    
    product_popularity[pid] = popularity

# Convert to weights
total_pop = sum(product_popularity.values())
product_weights = np.array([product_popularity[pid] for pid in product_ids])
product_weights = product_weights / product_weights.sum()

# ============================================================================
# GENERATE TRANSACTION DATA
# ============================================================================

print("  → Generating transactions with seasonality...")

transaction_data = []

# Calculate transactions per day with seasonality
days_range = (END_DATE - START_DATE).days
dates = [START_DATE + timedelta(days=i) for i in range(days_range)]

# Seasonality factors (more sales in festive months)
month_seasonality = {
    1: 0.85,   # January (post-holiday slump)
    2: 0.75,   # February (lowest)
    3: 0.80,   # March
    4: 0.90,   # April
    5: 0.95,   # May
    6: 1.00,   # June
    7: 1.05,   # July
    8: 1.10,   # August (festive season starts)
    9: 1.15,   # September
    10: 1.25,  # October (Diwali, festive)
    11: 1.35,  # November (Black Friday, sales)
    12: 1.40   # December (Christmas, New Year)
}

# Day of week factors (more on weekends)
day_of_week_factor = {
    0: 1.0,   # Monday
    1: 0.95,  # Tuesday
    2: 0.95,  # Wednesday
    3: 1.0,   # Thursday
    4: 1.1,   # Friday
    5: 1.25,  # Saturday
    6: 1.20   # Sunday
}

# Distribute transactions across dates
transactions_per_date = {}
for date in dates:
    base_trans = NUM_TRANSACTIONS / days_range
    seasonal_factor = month_seasonality[date.month]
    dow_factor = day_of_week_factor[date.weekday()]
    
    # Add random noise
    noise = np.random.normal(1.0, 0.15)
    
    num_trans = int(base_trans * seasonal_factor * dow_factor * noise)
    transactions_per_date[date] = max(1, num_trans)

# Adjust to get exactly NUM_TRANSACTIONS
current_total = sum(transactions_per_date.values())
adjustment_factor = NUM_TRANSACTIONS / current_total
transactions_per_date = {date: max(1, int(count * adjustment_factor)) 
                        for date, count in transactions_per_date.items()}

# Final adjustment
diff = NUM_TRANSACTIONS - sum(transactions_per_date.values())
if diff != 0:
    # Add/subtract from random dates
    dates_to_adjust = np.random.choice(list(transactions_per_date.keys()), 
                                      size=abs(diff), replace=True)
    for date in dates_to_adjust:
        transactions_per_date[date] += 1 if diff > 0 else -1

print(f"     ✓ Distributed across {len(dates)} days with seasonality")

# ============================================================================
# GENERATE INDIVIDUAL TRANSACTIONS
# ============================================================================

print("  → Creating transaction records...")

transaction_id = 1
batch_size = 10000
batches_completed = 0

for date, num_trans in transactions_per_date.items():
    # Sample customers for this day (with replacement - same customer can buy multiple times)
    day_customers = np.random.choice(customer_ids, size=num_trans, 
                                    p=customer_weights, replace=True)
    
    # Sample products for this day
    day_products = np.random.choice(product_ids, size=num_trans, 
                                   p=product_weights, replace=True)
    
    for cid, pid in zip(day_customers, day_products):
        # Get product details
        product = products_df[products_df['Product_ID'] == pid].iloc[0]
        
        # Quantity (most buy 1, some buy multiple)
        quantity_prob = np.random.random()
        if quantity_prob < 0.70:
            quantity = 1
        elif quantity_prob < 0.85:
            quantity = 2
        elif quantity_prob < 0.93:
            quantity = 3
        elif quantity_prob < 0.97:
            quantity = np.random.randint(4, 6)
        else:
            quantity = np.random.randint(6, 11)  # Bulk order
        
        # Calculate amount
        base_price = product['Price']
        discount = product['Discount_Percent']
        unit_price = base_price * (1 - discount / 100)
        
        # Add small random variation (taxes, rounding, coupons)
        price_variation = np.random.normal(1.0, 0.03)
        price_variation = np.clip(price_variation, 0.85, 1.15)
        
        amount = int(unit_price * quantity * price_variation)
        amount = max(10, amount)  # Minimum order value
        
        # Timestamp (add time to date)
        hour = int(np.random.beta(a=2, b=2) * 24)  # Peak around noon
        minute = np.random.randint(0, 60)
        second = np.random.randint(0, 60)
        timestamp = datetime.combine(date, datetime.min.time()) + \
                   timedelta(hours=hour, minutes=minute, seconds=second)
        
        # Payment method (realistic distribution for India)
        payment_methods = ['UPI', 'Credit Card', 'Debit Card', 'Net Banking', 
                          'Cash on Delivery', 'Wallet']
        payment_weights = [0.35, 0.20, 0.18, 0.12, 0.10, 0.05]
        payment_method = np.random.choice(payment_methods, p=payment_weights)
        
        # Transaction status (mostly successful)
        status_prob = np.random.random()
        if status_prob < 0.92:
            status = 'Completed'
        elif status_prob < 0.95:
            status = 'Pending'
        elif status_prob < 0.98:
            status = 'Cancelled'
        else:
            status = 'Refunded'
        
        # Delivery time (days) - only for completed orders
        if status == 'Completed':
            delivery_probs = [0.05, 0.15, 0.25, 0.25, 0.15, 0.10, 0.05]
            delivery_days = np.random.choice([1, 2, 3, 4, 5, 6, 7], p=delivery_probs)
        else:
            delivery_days = np.nan
        
        # Shipping address (same city or different)
        customer = customers_df[customers_df['Customer_ID'] == cid].iloc[0]
        if np.random.random() < 0.80:  # 80% ship to same city
            shipping_city = customer['City']
            shipping_state = customer['State']
        else:  # 20% ship to different city (gift, office, etc.)
            random_customer = customers_df.sample(1).iloc[0]
            shipping_city = random_customer['City']
            shipping_state = random_customer['State']
        
        # Coupon code (10% of transactions use coupons)
        if np.random.random() < 0.10:
            coupon_codes = ['SAVE10', 'FLASH20', 'FIRST50', 'WELCOME', 'SALE25']
            coupon_used = np.random.choice(coupon_codes)
            coupon_discount = np.random.choice([5, 10, 15, 20])
        else:
            coupon_used = None
            coupon_discount = 0
        
        # Rating given by customer (after delivery, not all rate)
        if status == 'Completed' and np.random.random() < 0.30:  # 30% rate
            # Biased towards product rating
            customer_rating = product['Rating'] + np.random.normal(0, 0.5)
            customer_rating = round(np.clip(customer_rating, 1, 5), 1)
        else:
            customer_rating = np.nan
        
        transaction_data.append({
            'Transaction_ID': f'TXN_{transaction_id:07d}',
            'Customer_ID': cid,
            'Product_ID': pid,
            'Transaction_Date': timestamp.date(),
            'Transaction_Time': timestamp.time(),
            'Quantity': quantity,
            'Unit_Price': int(unit_price),
            'Total_Amount': amount,
            'Payment_Method': payment_method,
            'Status': status,
            'Shipping_City': shipping_city,
            'Shipping_State': shipping_state,
            'Delivery_Days': delivery_days,
            'Coupon_Code': coupon_used,
            'Coupon_Discount': coupon_discount,
            'Customer_Rating': customer_rating
        })
        
        transaction_id += 1
        
        # Progress indicator
        if transaction_id % batch_size == 0:
            batches_completed += 1
            progress = (transaction_id / NUM_TRANSACTIONS) * 100
            print(f"     Progress: {progress:.1f}% ({transaction_id:,} / {NUM_TRANSACTIONS:,})")

# ============================================================================
# CREATE DATAFRAME
# ============================================================================

print("  → Assembling dataframe...")
transactions_df = pd.DataFrame(transaction_data)

# ============================================================================
# SAVE TO CSV
# ============================================================================

print("  → Saving to transactions.csv...")
transactions_df.to_csv('transactions.csv', index=False)

# ============================================================================
# SUMMARY STATISTICS
# ============================================================================

elapsed_time = time.time() - start_time

print("\n" + "="*70)
print("✅ TRANSACTIONS.CSV GENERATED SUCCESSFULLY!")
print("="*70)
print(f"\nTime taken: {elapsed_time:.2f} seconds")
print(f"Total transactions: {len(transactions_df):,}")
print(f"File size: {transactions_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

print("\n📊 DATA SUMMARY:")
print("-"*70)
print(f"Date range: {transactions_df['Transaction_Date'].min()} to {transactions_df['Transaction_Date'].max()}")
print(f"Amount range: ₹{transactions_df['Total_Amount'].min():,} - ₹{transactions_df['Total_Amount'].max():,}")
print(f"Average transaction: ₹{transactions_df['Total_Amount'].mean():,.0f}")
print(f"Total revenue: ₹{transactions_df['Total_Amount'].sum():,.0f}")

print("\n💳 PAYMENT METHOD DISTRIBUTION:")
print(transactions_df['Payment_Method'].value_counts())

print("\n📦 STATUS DISTRIBUTION:")
print(transactions_df['Status'].value_counts())

print("\n⭐ CUSTOMER RATINGS (when given):")
rating_dist = transactions_df['Customer_Rating'].dropna().value_counts().sort_index()
print(rating_dist)

print("\n🚚 DELIVERY TIME DISTRIBUTION:")
delivery_dist = transactions_df['Delivery_Days'].dropna().value_counts().sort_index()
print(delivery_dist)

print("\n📅 TRANSACTIONS BY MONTH (2024):")
transactions_2024 = transactions_df[transactions_df['Transaction_Date'] >= datetime(2024, 1, 1).date()]
if len(transactions_2024) > 0:
    monthly = pd.to_datetime(transactions_2024['Transaction_Date']).dt.month.value_counts().sort_index()
    print(monthly)

print("\n🎫 COUPON USAGE:")
coupon_usage = transactions_df['Coupon_Code'].notna().sum()
print(f"Transactions with coupons: {coupon_usage:,} ({coupon_usage/len(transactions_df)*100:.1f}%)")

print("\n⚠️ MISSING VALUES:")
missing = transactions_df.isnull().sum()
print(missing[missing > 0])

print("\n📋 SAMPLE DATA (first 5 rows):")
print(transactions_df[['Transaction_ID', 'Customer_ID', 'Product_ID', 
                       'Transaction_Date', 'Total_Amount', 'Status']].head())



GENERATING TRANSACTIONS.CSV

Generating 100,000 transactions...
  → Loading customers and products data...
     ✓ Loaded 10,000 customers
     ✓ Loaded 1,000 products
  → Modeling customer behavior patterns...
  → Calculating product popularity...
  → Generating transactions with seasonality...
     ✓ Distributed across 2191 days with seasonality
  → Creating transaction records...
     Progress: 10.0% (10,000 / 100,000)
     Progress: 20.0% (20,000 / 100,000)
     Progress: 30.0% (30,000 / 100,000)
     Progress: 40.0% (40,000 / 100,000)
     Progress: 50.0% (50,000 / 100,000)
     Progress: 60.0% (60,000 / 100,000)
     Progress: 70.0% (70,000 / 100,000)
     Progress: 80.0% (80,000 / 100,000)
     Progress: 90.0% (90,000 / 100,000)
     Progress: 100.0% (100,000 / 100,000)
  → Assembling dataframe...
  → Saving to transactions.csv...

✅ TRANSACTIONS.CSV GENERATED SUCCESSFULLY!

Time taken: 114.78 seconds
Total transactions: 100,000
File size: 57.09 MB

📊 DATA SUMMARY:
--------------